# EDDI19 - Workshop

## Setup

We first need to import the modules and data we will use through this notebook.

In [1]:
from IPython.display import clear_output

# User friendly functions

def loading(msg='Loading...'):
    print(msg)

def loaded(msg='Loaded!'):
    clear_output()
    print(msg)

loaded()

Loaded!


In [2]:
# Import needed modules
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Span, CDSView, \
RangeSlider, CustomJS, CustomJSFilter, DatetimeTickFormatter, \
HoverTool, PanTool, Legend, LabelSet
from bokeh.palettes import Spectral6, Category10_7
from bokeh.transform import factor_cmap
from bokeh.layouts import column, widgetbox, row

import calendar
from datetime import timedelta
from dateutil.relativedelta import relativedelta

output_notebook()
loaded('Modules imported!')

Modules imported!


In [3]:
# Function to get a csv as a DataFrame
def get_df(filename):
    return pd.read_csv(filename)

# Import CSVs containing the data
interactions = get_df('data/survey_interactions_2019_light.csv')
panelists = get_df('data/panelists_ndob.csv')
events = get_df('data/events.csv')
surveys = get_df('data/surveys.csv')

loaded('Data imported!')

Data imported!


## Data structure

In [4]:
interactions

,date,event_id,panelist_id,question_id,survey_id
0,2019-01-10 16:45:30,1211,17259,NaN,373
1,2019-01-10 16:52:24,1211,16849,NaN,373
2,2019-01-10 16:52:28,1211,15141,NaN,373
3,2019-01-10 16:54:34,1211,17468,NaN,373
4,2019-01-10 16:55:43,1211,17559,NaN,373
...,...,...,...,...,...
27144,2019-07-01 22:21:25,1207,14772,NaN,378
27145,2019-07-01 22:39:38,1211,15171,NaN,378
27146,2019-07-01 23:15:27,1211,16169,NaN,378
27147,2019-07-01 23:29:18,1207,16169,NaN,378


In [5]:
panelists

,id,birthdate,department,sex,panel,speed_test_average,leave_date,leave_reason
0,14225,1972-08-01,94,F,1,3963548,2018-03-22,exit
1,14226,1974-10-01,69,F,1,2752266,NaN,NaN
2,14227,1952-09-01,32,F,1,1142051,2017-04-28,exit
3,14228,1987-11-01,74,F,1,2407357,NaN,NaN
4,14229,1965-11-01,60,F,1,3425063,2018-11-21,exit
...,...,...,...,...,...,...,...,...
3551,17776,1956-03-01,69,M,2,0,2016-10-13,exit
3552,17777,1969-01-01,95,M,2,3647644,NaN,NaN
3553,17778,1966-02-01,13,F,2,5171952,NaN,NaN
3554,17779,1977-09-01,33,F,2,3628099,NaN,NaN


In [6]:
events

,id,codename,description
0,1081,create_absence,NaN
1,1083,end_absence,NaN
2,1084,end_survey,NaN
3,1154,phonecall_group_ci,NaN
4,1155,phonecall_group_ct,NaN
...,...,...,...
101,1294,ticket_open__sui,NaN
102,1295,ticket_open__surveys,NaN
103,1296,ticket_open__usage,NaN
104,1302,user_new_tablet,NaN


In [7]:
surveys

,id,alt_id,title,start_date,end_date,firm_end_date,estimated_time,invitation_count
0,306,NaN,Les relations entre générations,2014-02-06 15:00:00,2014-03-06 20:00:00,2014-05-31 12:00:00,25.0,984
1,307,NaN,"Enquête Santé, Travail et Environnement",2013-12-12 08:00:00,2014-02-06 20:00:00,2014-05-31 12:00:00,30.0,993
2,308,NaN,"Le couple, les enfants et la famille",2013-11-05 08:00:00,2013-12-10 20:00:00,2014-05-31 12:00:00,30.0,983
3,309,NaN,Environnement & Énergie,2013-10-03 12:00:00,2013-11-07 20:00:00,2014-05-31 12:00:00,30.0,997
4,310,NaN,"Loisirs culturels, médias et Internet",2013-06-06 08:00:00,2013-07-11 20:00:00,2014-05-31 12:00:00,30.0,1011
...,...,...,...,...,...,...,...,...
71,377,_201904_capsoc,Relations sociales et réseaux personnels,2019-04-25 12:00:00,2019-06-06 23:50:00,2019-06-13 23:50:00,30.0,2424
72,378,_201905_NatInterest,Naturbanisation,2019-05-23 12:00:00,2019-06-30 23:50:00,2019-07-01 23:50:00,10.0,2405
73,379,_201906_psysoc,Culture et personnalité,2019-05-23 12:00:00,2019-06-30 23:50:00,2019-07-01 23:50:00,20.0,2405
74,380,_201905_cee,Questions d'actualité après les élections euro...,2019-05-27 08:00:00,2019-06-30 23:50:00,2019-07-01 23:50:00,20.0,2405


## Data visualization

### Panelist count by gender

Here is a first simple chart showing the number of men and women in the ELIPSS panel.

In [8]:
loading()

s = panelists.sex.value_counts()

genders = list(s.index)
counts = list(s.values)

source = ColumnDataSource(data=dict(genders=genders, counts=counts))

p = figure(
    x_range=genders,
    title="Panelist count by gender",
    plot_height=400,
    plot_width=400,
)

p.vbar(
    x='genders',
    top='counts',
    width=0.9,
    source=source,
    fill_color=factor_cmap('genders', palette=Spectral6, factors=genders),
)

p.add_layout(
    LabelSet(
        x='genders',
        y='counts',
        text='counts',
        level='glyph',
        x_offset=-13.5,
        y_offset=0,
        source=source,
        render_mode='canvas'
    )
)

p.xgrid.grid_line_color = None
p.y_range.start = 0

loaded()
show(p)

Loaded!


### Survey start date by age

We will now see a more complex chart showing the start dates of a survey by the panelists and by age, after the survey was published.

In [9]:
loading()

# Gets the id of events 'survey_start'
EVENT_START_IDS = events[events['codename'].isin(['survey_start'])]['id']
# Remove interactions that are not 'survey_start'
iactions = interactions[interactions['event_id'].isin(EVENT_START_IDS)]

# Adds birthdate of each panelists on each interaction
iactions = iactions.merge(
    panelists[['id', 'birthdate']],
    left_on='panelist_id', right_on='id'
).drop(columns=['id'])

# Change column `date` and `birthdate` type
iactions[['date', 'birthdate']] = iactions[['date', 'birthdate']].apply(pd.to_datetime)

# Adds `age` column
def get_age(date):
    now = pd.to_datetime('now')
    days_in_year = 366 if calendar.isleap(now.year) else 365
    day_of_year = int(now.strftime("%j"))
    delta = relativedelta(now, date)
    date_day_of_year = int(date.strftime("%j"))
    countdown = (date_day_of_year-day_of_year)
    if countdown >= 0:
        return delta.years - countdown/days_in_year + 1
    else:
        return delta.years - countdown/days_in_year
iactions['age'] = iactions['birthdate'].apply(get_age)


# Change column `start_date` type
surveys[['start_date']] = surveys[['start_date']].apply(pd.to_datetime)

# Get survey start dates
survey_ids = iactions['survey_id'].unique()
survey_start_dates = surveys[surveys['id'].isin(survey_ids)][['id', 'start_date']].rename(
    columns={'id':'survey_id'}
)

iactions[['survey_id']] = iactions[['survey_id']].astype(str)
source = ColumnDataSource(iactions)

p = figure(
    title="Survey start date by age",
    x_axis_type='datetime',
    sizing_mode = 'scale_width'
)

cmap = factor_cmap('survey_id', palette=Category10_7, factors=[str(s) for s in survey_ids])

p.circle(
    'date',
    'age',
    source=source,
    fill_alpha=0.25,
    line_alpha=0,
    fill_color=cmap,
    size=6
)

transform = cmap['transform']
cmap_dict = dict(zip(transform.factors, transform.palette))
for index, row in survey_start_dates.iterrows():
    survey_id = str(row['survey_id'])
    p.add_layout(
        Span(
            location=row['start_date'],
            dimension='height',
            line_color=cmap_dict[survey_id]
        )
    )

loaded()
show(p)

Loaded!


### Response time distribution by survey

Here is an even more complex chart showing the response time distribution by survey.

First, we need to process the data of the survey events:

In [10]:
loading()

def seconds_to_datetime(df):
    return timedelta(seconds=df)

def get_completion_time(df):
    dates = list(df['date'])
    if len(dates) == 2:
        seconds = abs((dates[0]-dates[1]).total_seconds())
        df['completion_time'] = seconds
    return df

def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)


iactions2 = interactions

# Cast dates
iactions2[['date']] = iactions2[['date']].apply(pd.to_datetime)

# Get completion times
iactions2 = iactions2.groupby(['panelist_id', 'survey_id']).apply(get_completion_time)

# Remove unused columns
iactions2 = iactions2[['completion_time', 'panelist_id', 'survey_id']].drop_duplicates()
iactions2 = iactions2[iactions2['completion_time'].notnull()]

# Calc stats
iactions2 = iactions2.groupby(['survey_id']).agg({'completion_time': ['mean', 'min', q1, 'median', q3, 'max']})

# Simplify stat column indexes
iactions2.columns = iactions2.columns.get_level_values(1)

# Merge survey information
iactions2 = iactions2.merge(surveys[['id', 'title', 'estimated_time']], left_on='survey_id', right_on='id')

# Cast stats from seconds to datetime
stats_col = ['mean', 'min', 'q1', 'median', 'q3', 'max']
iactions2[stats_col] = iactions2[stats_col].apply(lambda x: pd.to_timedelta(x, unit='seconds'))
iactions2[['estimated_time']] = iactions2[['estimated_time']].apply(lambda x: pd.to_timedelta(x, unit='minutes'))

loaded('Data processed!')

Data processed!


Then, we can set up the complex plot using Bokeh:

In [11]:
loading()

df = iactions2
df['id'] = df['id'].astype(str)
source = ColumnDataSource(df)

ids = df['id']
rng_start = ids.min()
rng_end = ids.max()
rng_value = (rng_start, rng_end)

p = figure(
    title="Response time distribution by survey",
    x_axis_type='datetime',
    y_range=[i for i in ids.values],
    tools=['save', 'reset'],
    sizing_mode = 'scale_width'
)

p.x_range.start = 0
p.x_range.end = pd.to_timedelta('01:15:00').total_seconds()*1000
p.x_range.bounds = (0, None)
p.xaxis.formatter = DatetimeTickFormatter(minutes=['%M min'], hourmin='%H h %M min')
p.ygrid.grid_line_color = None
p.xgrid.grid_line_dash = 'dashed'

# Spread between 25th and 50th percentiles
q2 = p.hbar(y='id', left='q1', right='median', height=0.8, color='limegreen', alpha=0.5, source=source)

# Spread between 50th and 75th percentiles
q3 = p.hbar(y='id', left='median', right='q3', height=0.8, color='indianred', alpha=0.5, source=source)

p.add_tools(HoverTool(
    renderers=[q2, q3],
    formatters={key: 'datetime' for key in ['mean', 'min', 'q1', 'median', 'q3', 'max', 'estimated_time']},
    tooltips=[
        ('Survey', '@title'),
        ('Estimated time', '@estimated_time{%H:%M:%S}'),
        ('Minimum', '@min{%H:%M:%S}'),
        ('25%', '@{q1}{%H:%M:%S}'),     
        ('Median', '@{median}{%H:%M:%S}'),
        ('75%', '@{q3}{%H:%M:%S}'),
        ('Maximum', '@max{%H:%M:%S}')
    ],
    point_policy='follow_mouse'
))

# Whiskers
p.rect(x='min', y='id', height=0.8, width=0.01, color='gray', source=source)
p.rect(x='max', y='id', height=0.8, width=0.01, color='gray', source=source)

# Stems
s = p.segment(x0='min', y0='id', x1='max', y1='id', color='gray', source=source)

# Expected completion time
e = p.circle(x='estimated_time', y='id', size=8, color="black", source=source)

# Legend
legend = Legend(items=[
    ('Estimated time', [e]),
    ('25%-50%', [q2]),
    ('50%-75%', [q3]),
    ('min-max', [s]),
], location=(10, 300))
p.add_layout(legend, 'right')

# Horizontal panning
p.add_tools(
    PanTool(dimensions="width"),
)

loaded()
show(p)

Loaded!
